In [2]:
#done
pip install requests python-dotenv selenium bs4 web3 pymongo psycopg2 pandas textblob  python-dotenv


   ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
   -------- ------------------------------- 2.1/9.5 MB 10.7 MB/s eta 0:00:01
   --------------- ------------------------ 3.7/9.5 MB 9.1 MB/s eta 0:00:01
   ------------------------- -------------- 6.0/9.5 MB 10.0 MB/s eta 0:00:01
   ----------------------------------- ---- 8.4/9.5 MB 10.6 MB/s eta 0:00:01
   ---------------------------------------- 9.5/9.5 MB 10.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 1.4/1.4 MB 8.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/882.3 kB ? eta -:--:--
   --------------------------------------- 882.3/882.3 kB 10.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 1.2/1.2 MB 9.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/624.3 kB ? eta -:--:--
   --------------------------------

In [20]:
import os
from dotenv import load_dotenv
# Load variables from .env file
load_dotenv() 
# Access environment variables
api_key = os.getenv("COINGECKO_API_KEY")
print(api_key)  # This should print your API key


CG-3rbimK36SKcemnZUzSMkoptC


In [3]:
print('.')

.


In [ ]:
import os
import logging
import time
import requests
from dotenv import load_dotenv
from functools import wraps
from datetime import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from web3 import Web3
from pymongo import MongoClient
import psycopg2
import pandas as pd
from textblob import TextBlob  

In [ ]:
import os
import logging
import time
import requests
from dotenv import load_dotenv
from functools import wraps
from datetime import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from pymongo import MongoClient
import psycopg2
from textblob import TextBlob  

# Load environment variables
load_dotenv()

# Configure logging
logger = logging.getLogger("CryptoDataCollector")
logger.setLevel(logging.INFO)
handler = logging.StreamHandler()
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

# Custom exception class
class APIError(Exception):
    pass

class Config:
    API_KEYS = {
        'coingecko': os.getenv('COINGECKO_API_KEY')
    }
    
    DB_CONFIG = {
        'postgres': os.getenv('POSTGRES_URL'),
        'mongo': os.getenv('MONGO_URL')
    }
    
    @classmethod
    def get_rate_limits(cls):
        return {
            'coingecko': 50
        }

class APIClientBase:
    def __init__(self, name):
        self.name = name
        self.base_url = None
        self.rate_limit = Config.get_rate_limits().get(name, 10)
        self.last_request = 0
    
    def rate_limiter(func):
        @wraps(func)
        def wrapper(self, *args, **kwargs):
            retries = 3
            while retries > 0:
                elapsed = time.time() - self.last_request
                if elapsed < 60 / self.rate_limit:
                    time.sleep(60 / self.rate_limit - elapsed)
                try:
                    response = func(self, *args, **kwargs)
                    if response:
                        self.last_request = time.time()
                        return response
                except APIError as e:
                    logger.error(f"API Error: {str(e)}")
                retries -= 1
                time.sleep(2 ** (3 - retries))
            raise APIError("Failed after multiple retries")
        return wrapper

    def handle_errors(self, response):
        if response.status_code != 200:
            logger.error(f"API Error: {response.status_code} - {response.text}")
            raise APIError(f"API Error: {response.status_code}")

class CoinGeckoClient(APIClientBase):
    def __init__(self):
        super().__init__('coingecko')
        self.base_url = 'https://api.coingecko.com/api/v3/'

    @APIClientBase.rate_limiter
    def get_real_time_prices(self, coin_ids):
        """
        Gets real-time price data for the specified coins.
        coin_ids should be a list of coin ids as defined in the CoinGecko API (e.g., ['bitcoin', 'ethereum']).
        """
        url = f"{self.base_url}simple/price"
        params = {
            'ids': ','.join(coin_ids),
            'vs_currencies': 'usd'
        }
        response = requests.get(url, params=params)
        self.handle_errors(response)
        return response.json()

class CryptoScraper:
    def __init__(self, headless=True):
        self.options = webdriver.FirefoxOptions()
        if headless:
            self.options.add_argument("--headless")
        self.driver = webdriver.Firefox(options=self.options)

    def scrape_news(self, url):
        try:
            self.driver.get(url)
            WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "article-content"))
            )
            soup = BeautifulSoup(self.driver.page_source, 'html.parser')
            title = soup.find('h1').text if soup.find('h1') else "No Title"
            paragraphs = soup.find_all('p')
            content = ' '.join(p.text for p in paragraphs) if paragraphs else "No Content"
            return {
                'title': title,
                'content': content,
                'sentiment': self.analyze_sentiment(content)
            }
        except Exception as e:
            logger.error(f"Scraping error: {str(e)}")
            return None

    def analyze_sentiment(self, text):
        analysis = TextBlob(text)
        return {
            'polarity': analysis.sentiment.polarity,
            'subjectivity': analysis.sentiment.subjectivity
        }

class PostgreSQLConnector:
    def __init__(self):
        self.conn = psycopg2.connect(Config.DB_CONFIG['postgres'])
    
    def store_market_data(self, data):
        query = """
        INSERT INTO market_data (symbol, price, volume, timestamp)
        VALUES (%s, %s, %s, %s)
        """
        try:
            with self.conn.cursor() as cur:
                cur.executemany(query, [
                    (d['symbol'], d['price'], d.get('volume', None), datetime.utcnow())
                    for d in data
                ])
            self.conn.commit()
        except Exception as e:
            self.conn.rollback()
            logger.error(f"Database error: {str(e)}")

class MongoDBConnector:
    def __init__(self):
        self.client = MongoClient(Config.DB_CONFIG['mongo'])
        self.db = self.client.crypto_data
    
    def store_news_article(self, article):
        collection = self.db.news_articles
        article['timestamp'] = datetime.utcnow()
        return collection.insert_one(article)

class CryptoDataCollector:
    def __init__(self):
        self.gecko = CoinGeckoClient()
        self.scraper = CryptoScraper()
        self.pg = PostgreSQLConnector()
        self.mongo = MongoDBConnector()
        
    def collect_all_data(self):
        try:
            # Fetch real-time prices from CoinGecko
            prices = self.gecko.get_real_time_prices(['bitcoin', 'ethereum'])
            processed = self._process_price_data(prices)
            self.pg.store_market_data(processed)
            # Scrape news from a crypto news site
            news = self.scraper.scrape_news('https://cointelegraph.com/')
            if news:
                self.mongo.store_news_article(news)
        except Exception as e:
            logger.error(f"Collection error: {str(e)}")
            raise

    def _process_price_data(self, raw_data):
        processed = []
        for coin, info in raw_data.items():
            processed.append({
                'symbol': coin,
                'price': info.get('usd'),
                'volume': None  # CoinGecko's simple price API does not return volume data.
            })
        return processed

if __name__ == "__main__":
    collector = CryptoDataCollector()
    while True:
        collector.collect_all_data()
        time.sleep(300)